In [41]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%autoreload 2

In [43]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
from collections import Counter
import pandas as pd
import numpy as np

'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation, Neptune_data_read
from parameter import *
from dspike_formulas import *



In [44]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2017-09-13_DS_calib/"
files_start = 8327
files_end = 8328

In [45]:
spikeSn_ls = [["117"],["118", "122", "124"]]
spike_dict_117_122_meas = {"118": 0.010295, "119": 0.009073, "120" : 0.033863, "122" : 0.806799, "124" : 0.011484} # DS compositon 1ppm - 15.09.17

meas_dict = { "118" : 3.157775, "119" : 1.119727, "120" : 4.250492, "122" : 0.603920, "124" : 0.754685} # mean of 1ppm NIST Std used for calib-15.09.17

In [46]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["117", "118", "119", "120", "122", "124"]
cup_config = cycles_spike

# Mass Range of cup configuration
mass_range = cycle_Sn_spike_mass_range

# Isotopes used for Interference correction
corr_isotopes_2 = {"Te" : "125", "Xe" : "129"}
#corr_isotopes_2 = {}
#corr_isotopes_Sb = {"Te": "125"}

denom_isotope = "117"
Sn_monitor = ["125"]

#def eval_iso_list(isotopes_list, denom_isotope, monitor_iso):
#    isotope_den = denom_isotope
#    isotopes_list.remove(isotope_den)
#    for i in range(len(monitor_iso)):
#        isotopes_list.append(monitor_iso[i])
#    isotopes_list.sort()
#   return isotopes_list

#isotope_ls = eval_iso_list(Sn_isotopes, denom_isotope, Sn_monitor)

#data_sample_column = [(i + "/" + denom_isotope) for i in isotope_ls]    
isotopes = [["118", "122", "124"]]

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = False

In [47]:
isotopes

[['118', '122', '124']]

In [48]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
#files_1 = range(5552,5623,1) + range(5632,5729, 1)

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_zero_1.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)

In [49]:
df_zero_1

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,
2017-09-15 22:21:00,1,DS-NIST Mix 1ppm opti.,0.000528,0.000671,0.000085,-2.427000e-05,7.380700e-06,1.973827,-0.000391,9.651253,0.000138,10.279499,2.676045,7.303507,11.796593
2017-09-15 22:21:00,2,DS-NIST Mix 1ppm opti.,0.000529,0.000677,0.000091,-7.134800e-06,-7.893300e-06,1.962916,-0.000372,9.599029,0.000126,10.224110,2.661536,7.264666,11.733966
2017-09-15 22:21:00,3,DS-NIST Mix 1ppm opti.,0.000533,0.000675,0.000104,-1.594870e-05,1.087390e-05,1.961641,-0.000382,9.593037,0.000144,10.217846,2.659962,7.260296,11.726680
2017-09-15 22:21:00,4,DS-NIST Mix 1ppm opti.,0.000545,0.000682,0.000090,-1.647170e-05,2.244100e-05,1.956707,-0.000373,9.568941,0.000146,10.192323,2.653390,7.242544,11.698239
2017-09-15 22:21:00,5,DS-NIST Mix 1ppm opti.,0.000553,0.000677,0.000094,-2.506910e-05,-5.588700e-06,1.924119,-0.000308,9.411070,0.000148,10.023988,2.609631,7.123811,11.506335
2017-09-15 22:21:00,6,DS-NIST Mix 1ppm opti.,0.000536,0.000685,0.000087,-3.370080e-05,1.169750e-05,1.970677,-0.000390,9.636971,0.000148,10.264235,2.672135,7.292945,11.779246
2017-09-15 22:21:00,7,DS-NIST Mix 1ppm opti.,0.000561,0.000694,0.000090,-1.327790e-05,-3.056200e-06,1.971988,-0.000396,9.642710,0.000154,10.270397,2.673400,7.297141,11.785786
2017-09-15 22:21:00,8,DS-NIST Mix 1ppm opti.,0.000571,0.000692,0.000080,-1.814370e-05,3.739900e-06,1.962236,-0.000366,9.595094,0.000152,10.220624,2.660650,7.262261,11.729098
2017-09-15 22:21:00,9,DS-NIST Mix 1ppm opti.,0.000578,0.000874,0.000478,-6.975416e-04,2.399435e-04,1.886012,0.000084,8.333716,-0.000040,9.753940,2.576241,6.470032,11.171728


In [25]:
# For one datapoint
files_1 = 7208

df = NU_data_read(path, sample, cup_config)
cycles = range(1, len(df.data_read(sample).index)+1)

df_zero = df.data_zero_corr(sample)
new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
#new_corr.line2_corr(df_zero, "119")

data_sample = new_corr.raw_ratios(denom_isotope)
data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
df_zero

{'cycle1': {'Ax (1)': Counter({1: 0.00013836589999999954,
           2: 0.00012571850000000009,
           3: 0.00014383660000000069,
           4: 0.00014579099999999928,
           5: 0.00014788939999999841,
           6: 0.00014835819999999868,
           7: 0.00015420509999999887,
           8: 0.00015245509999999886,
           9: -4.0447400000000883e-05,
           10: 0.00014881189999999857,
           11: 0.00012035759999999909,
           12: 0.00014608449999999919,
           13: 0.00012900490000000014,
           14: 0.0003078537999999997,
           15: 0.00014112969999999919,
           16: 0.00014536620000000014,
           17: 0.00013632270000000016,
           18: 0.00014228219999999972,
           19: 0.00012344829999999876,
           20: 0.00012890919999999917}),
  'H1 (1)': Counter({1: 9.6512528109999991,
           2: 9.5990288110000002,
           3: 9.5930372110000004,
           4: 9.5689413110000014,
           5: 9.4110702110000002,
           6: 9.63697051100

,118,122,124
0,0.611626,0.868462,0.166942
1,0.611635,0.868524,0.166965
2,0.611610,0.868468,0.166942
3,0.611635,0.868500,0.166952
4,0.611632,0.868459,0.166941
5,0.611592,0.868393,0.166938
6,0.611631,0.868470,0.166954
7,0.611604,0.868456,0.166944
8,0.611619,0.868482,0.166959
9,0.611635,0.868453,0.166939


In [39]:
#Spike Calculation uncorrected
data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_sample, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
data_spike_calc = data_spike_obj.dspike_corr(3, 6, -0.1, -2.2, 'x')
beta = data_spike_calc["frac_ins_x2.5"]


In [40]:
data_spike_calc

,n0:x,n0:y,n0:z,N0:x,N0:y,N0:z,a_nat0,b_nat0,c_nat0,m0.0:x,...,e_nat2,f_nat2,g_nat2,a_ins2,b_ins2,c_ins2,Nr2:x,Nr2:y,Nr2:z,frac_nat_x2
0,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619120,...,-4.404106,1.735338,-0.293320,0.251476,0.240987,-0.185533,3.157276,0.603451,0.753870,-0.018563
1,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619114,...,-4.403811,1.735305,-0.293300,0.251452,0.240961,-0.185511,3.157215,0.603393,0.753770,-0.020860
2,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619126,...,-4.403665,1.735289,-0.293291,0.251440,0.240954,-0.185506,3.157184,0.603364,0.753720,-0.022005
3,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619114,...,-4.403773,1.735301,-0.293298,0.251449,0.240955,-0.185507,3.157207,0.603385,0.753757,-0.021162
4,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619121,...,-4.403271,1.735245,-0.293264,0.251407,0.240918,-0.185477,3.157101,0.603286,0.753585,-0.025082
5,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619135,...,-4.403647,1.735287,-0.293289,0.251439,0.240958,-0.185509,3.157180,0.603360,0.753714,-0.022141
6,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619148,...,-4.403785,1.735302,-0.293299,0.251451,0.240974,-0.185523,3.157209,0.603387,0.753761,-0.021066
7,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619166,...,-4.403742,1.735297,-0.293296,0.251447,0.240975,-0.185523,3.157200,0.603379,0.753746,-0.021404
8,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.579143,...,-5.048146,1.804461,-0.335065,0.305596,0.271258,-0.210513,3.288070,0.736611,0.994120,4.753326
9,3.157775,0.60392,0.754685,3.15509,0.601396,0.750307,0.325585,1.387873,-1.111603,0.619386,...,-4.399032,1.734778,-0.292982,0.251068,0.240709,-0.185305,3.156212,0.602450,0.752134,-0.058202


,118,122,124,117,125,126,129
0,1.904714,2.704546,0.519888,3.114180,0.000168,0.000370,0.010430
1,2.029933,2.882513,0.554133,3.318864,0.000190,0.000386,0.010470
2,2.021766,2.870847,0.551852,3.305644,0.000189,0.000381,0.010468
3,1.995586,2.833662,0.544716,3.262708,0.000184,0.000369,0.010459
4,1.956013,2.777350,0.533880,3.198021,0.000190,0.000378,0.010467
5,2.003260,2.844407,0.546803,3.275485,0.000201,0.000383,0.010464
6,1.978430,2.809226,0.540043,3.234682,0.000196,0.000367,0.010439
7,1.951028,2.770391,0.532555,3.190018,0.000194,0.000354,0.010452
8,2.038959,2.895265,0.556592,3.333708,0.000188,0.000390,0.010461
9,2.044290,2.902661,0.557967,3.342334,0.000205,0.000401,0.010485


In [18]:
#Interference correction based on beta from first DS-Inversion
cup_config_x = {"cycle1" : {"129" : "129", "126" : "126", "125" : "125", "124" : "124", "122" : "122", "118" : "118", "117" : "117"}}
mass_range_x = Isotopes_Mass_Range()
mass_range_x.add_mass_range_dict({"117" : ["Sn"], "118" : ["Sn"], "122" : ["Sn", "Te"], "124" : ["Sn", "Te", "Xe"], "125" : ["Te"], "126": ["Te", "Xe"], "129" : ["Xe"]})
data_corr = {}
data_corr_raw = {}
cycle_no = 20
cycles = range(0, len(data_sample.index))

for n in cycles:
    beta_x = beta[n]
    spike_corr = int_norm(data_spike_calc, n, cup_config_x, database, mass_range_x, corr_isotopes_2)
    
    data_corr[n] = {}
    data_corr_raw[n] = {}
    for isotope in isotopes[0]:
        # interference corrected ratios
        data_corr[n][isotope] = spike_corr.interference_corr_ratio("Sn", isotope , "117", beta[n])
        #use r = R(m2/m1)^-beta -> raw ratios for next double spike inversion
        mass_isotope_nom = spike_corr.database["Sn"]["Masses"].get_Isotope_mass(isotope) #e.g. M(116Sn)
        mass_isotope_denom = spike_corr.database["Sn"]["Masses"].get_Isotope_mass("117")# e.g. M(120Sn)
        data_corr_raw[n][isotope] = data_corr[n][isotope] * (mass_isotope_nom/mass_isotope_denom)**(-beta[n])
        

KeyError: 'n0:x'

In [ ]:
pd.DataFrame.from_dict(data_corr_raw, orient = 'index')

In [79]:
element_denom = "Sn"
isotope_denom = "117"
beta_x = beta[0]

corr_dict = {}
counter = 0
for level in mass_range_x.get_order_of_corr(corr_isotopes_2):
    if counter == 0:
        for isotope in level:
            element_nom = mass_range_x.get_isotopes(isotope)[0]
            corr_dict[isotope] = spike_corr.int_norm(element_nom, element_denom, isotope, isotope_denom, beta)
        counter += 1
    elif counter > 0:
        for isotope in level:
            corr_isotopes = list(mass_range_x.get_graph_of_corr(corr_isotopes_2)[isotope])
            if (len(corr_isotopes) > 1):
                for mass_no in corr_isotopes:
                    if mass_no in mass_range_x.get_order_of_corr(corr_isotopes_2)[counter-1]:
                        corr_isotope = mass_no
            else:
                corr_isotope = corr_isotopes[0]

            element_nom = set(mass_range_x.get_isotopes(isotope))
            element_nom = element_nom.difference(mass_range_x.get_isotopes(corr_isotope))
            element_nom = list(element_nom)[0]
            element_corr = set(mass_range_x.get_isotopes(corr_isotope))
            element_corr = element_corr.intersection(mass_range_x.get_isotopes(isotope))
            element_corr = list(element_corr)
            
            if (len(element_corr) > 1):
                print element_corr
                corr_dict[isotope] = spike_corr.int_norm(element_nom, element_denom, isotope, isotope_denom, beta)
                for element in element_corr:
                    corr_isotope_x = corr_isotopes_2[element]
                    print isotope
                    print corr_isotope_x
                    true_ratio_corr = database[element]["Ratios"].get_ratio(isotope, corr_isotope_x)
                    print true_ratio_corr
                    corr_dict[isotope] -= corr_dict[corr_isotope_x] * true_ratio_corr
                    print corr_dict[corr_isotope_x]
                    print corr_dict[isotope]
            else:
                print isotope
                corr_dict[isotope] = spike_corr.int_norm(element_nom, element_denom, isotope, isotope_denom, beta)
                print corr_isotope
                true_ratio_corr = database[element_corr[0]]["Ratios"].get_ratio(isotope, corr_isotope)
                corr_dict[isotope] -= corr_dict[corr_isotope] * true_ratio_corr
                
            
        counter += 1

122
125
124
125
126
129


In [80]:
mass_range_x.get_graph_of_corr(corr_isotopes_2)

{'117': set(),
 '118': set(),
 '122': {'125'},
 '124': {'125', '129'},
 '125': set(),
 '126': {'129'},
 '129': set()}

In [81]:
mass_range_x.get_order_of_corr(corr_isotopes_2)

[{'117', '118', '125', '129'}, {'122', '124', '126'}]

In [67]:
graph = {}
for mass_no in mass_range_x.isotopes_mass_range:
    print mass_no
    isotopes = mass_range_x.isotopes_mass_range[mass_no]
    graph[mass_no] = set()
    for isotope in isotopes:
        print isotope
        if isotope in corr_isotopes_2 and set(isotopes).issubset(corr_isotopes_2) == False:
            if isotope == mass_range_x.isotopes_mass_range[mass_no][0]:
                None
            else:
                print mass_no
                graph[mass_no].add(corr_isotopes_2[isotope])
            #print isotope
            #print set(isotopes).issubset(corr_isotopes_2)
            #print corr_isotopes_2[isotope]
        elif isotope not in corr_isotopes_2:
            None
        elif len(mass_range_x.isotopes_mass_range[corr_isotopes_2[isotope]]) == 1 and len(mass_range_x.isotopes_mass_range[mass_no]) > 1:
            if isotope == mass_range_x.isotopes_mass_range[mass_no][0]:
                None
            else:
                print mass_no
                graph[mass_no].add(corr_isotopes_2[isotope])

graph

122
Sn
Te
122
117
Sn
124
Sn
Te
124
Xe
124
125
Te
126
Te
Xe
126
129
Xe
118
Sn


{'117': set(),
 '118': set(),
 '122': {'125'},
 '124': {'125', '129'},
 '125': set(),
 '126': {'129'},
 '129': set()}

In [90]:
data_corr_raw = pd.DataFrame.from_dict(data_corr_raw, orient = 'index')
#pd.DataFrame.from_dict(data_corr, orient = 'index')
data_corr_raw

,122,124,118
0,0.868448,0.166905,0.611626
1,0.868511,0.166929,0.611635
2,0.868455,0.166906,0.611610
3,0.868487,0.166917,0.611635
4,0.868445,0.166904,0.611632
5,0.868379,0.166902,0.611592
6,0.868457,0.166918,0.611631
7,0.868444,0.166909,0.611604
8,0.868469,0.166923,0.611619
9,0.868440,0.166902,0.611636


In [91]:
data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_corr_raw, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
data_spike_calc_2 = data_spike_obj.dspike_corr(3, 6, -0.1, -2.2, 'x')
data_spike_calc_2["frac_ins_x2.5"]

0    -2.200630
1    -2.202190
2    -2.200687
3    -2.201707
4    -2.200622
5    -2.198177
6    -2.200656
7    -2.200200
8    -2.200817
9    -2.200494
10   -2.199512
11   -2.199293
12   -2.200679
13   -2.202520
14   -2.199945
15   -2.200194
16   -2.201658
17   -2.198440
18   -2.200815
19   -2.199085
Name: frac_ins_x2.5, dtype: float64

In [22]:
data_spike_calc_2.to_csv(path+"test_2.csv")

In [47]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample = True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        new_corr.line2_corr(df_zero, "119")
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        new_corr.line2_corr(df_zero, "119")
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    data_sample = new_corr.raw_ratios(denom_isotope)
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_sample, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
    data_spike_calc = data_spike_obj.dspike_corr(3, 6, 0.0069, -2.24, 'z')
    data_spike_calc = new_corr.mad_outlier_rejection(data_spike_calc)
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_spike_calc["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_spike_calc["sample"] = sample_name
    data_spike_calc["cycle"] = data_sample.index.values
    
    #data_sample_raw["cycle"] = data_sample_raw.index.values
    #data_sample = data_sample.merge(data_sample_raw)
    
    #df_internal_norm = df_internal_norm.append(data_sample)

KeyError: 0

In [6]:
spikeCd_ls = [["111"],["113", "112", "114"]]

In [7]:
Cd_masses ={"110" : 109.903007, "111" : 110.904182, "112" : 111.9027577,
            "113" : 112.9044014, "114" : 113.9033586, "116" : 115.904756}

Cd_spike = {"112" : 0.025899, "113" : 0.470870, "114" : 0.020021}
Cd_spike_denom = "111"

Cd_std = {"112" : 1.878410, "113" : 0.950024, "114" : 2.227540}
Cd_std_denom = "111"

In [8]:
# Masses object used for calc
Cd_mass_obj = load_mass_dict(Cd_masses)

# Abundance object used for calc
Cd_std_obj = load_ratio_dict(Cd_std, Cd_std_denom)
Cd_spike_obj = load_ratio_dict(Cd_spike, Cd_spike_denom)

In [9]:
sample_1567 = calc_dspike_sample(Cd_std_obj,df_new,Cd_spike_obj,Cd_mass_obj,spikeCd_ls ,"111")
sample_1567_res = sample_1567.dspike_corr(3, 6, 0.0069, -2.24, 'z')
print sample_1567_res["Nr2:z"].mean()
print ((sample_1567_res["Nr2:z"].std()/np.sqrt(60))/sample_1567_res["Nr2:z"].mean())*10**6
print 2.227540 * (113.9033586/110.904182)** sample_1567_res["frac_nat_z2"].mean()
#sim1_mix.to_csv(path+'/Spike_Sim/'+"sim_117_122,120 d=117, q=0.48, p=0.36.csv")

2.2246218983
82.0132881511
2.22462145682


In [10]:
sample_1567_res

,n0:x,n0:y,n0:z,N0:x,N0:y,N0:z,a_nat0,b_nat0,c_nat0,m0.0:x,...,e_nat2,f_nat2,g_nat2,a_ins2,b_ins2,c_ins2,Nr2:x,Nr2:y,Nr2:z,frac_nat_z2
0,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568895,...,-1.097095,3.528774,-4.400945,0.728247,1.002542,-0.348854,0.949261,1.877653,2.224869,-0.044968
1,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568993,...,-1.096460,3.527418,-4.398398,0.727553,1.002015,-0.348513,0.948525,1.876923,2.222294,-0.088360
2,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568787,...,-1.096962,3.528490,-4.400411,0.727736,1.002526,-0.348612,0.949106,1.877500,2.224329,-0.054064
3,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568861,...,-1.097353,3.529325,-4.401979,0.727926,1.002912,-0.348712,0.949560,1.877949,2.225915,-0.027355
4,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568773,...,-1.097449,3.529529,-4.402362,0.728077,1.002979,-0.348785,0.949670,1.878059,2.226303,-0.020824
5,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568886,...,-1.096882,3.528320,-4.400091,0.727803,1.002420,-0.348641,0.949014,1.877408,2.224005,-0.059512
6,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568953,...,-1.097162,3.528917,-4.401212,0.727907,1.002704,-0.348698,0.949338,1.877730,2.225139,-0.040411
7,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568865,...,-1.096381,3.527249,-4.398080,0.727437,1.001957,-0.348457,0.948433,1.876832,2.221973,-0.093779
8,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568963,...,-1.097087,3.528757,-4.400913,0.727976,1.002603,-0.348727,0.949252,1.877644,2.224837,-0.045502
9,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568902,...,-1.097153,3.528897,-4.401175,0.727960,1.002680,-0.348722,0.949327,1.877719,2.225102,-0.041035
